In [ ]:
#! pip install "altair[all]"
import pandas as pd
import altair as alt
from pathlib import Path

In [ ]:
chatbgc_version	= "0.2.1-alpha"
benchmark_version = "v0.2.1"

In [ ]:
df = pd.read_csv(f"../result/benchmark_result_{chatbgc_version}_{benchmark_version}.tsv", sep="\t", index_col=0)

In [ ]:
df.loc[:, ["id", "question", "difficulty", "model", "llm_type"]]

In [ ]:
df_question = pd.read_json("../test/question_sql_pair.json")
df.loc[:, "sql_query_success"] = True
avalaible_models = {"ollama" : ["llama3.1_70b", "llama3.1_8b",
                                "mistral-nemo", "mistral_7b", 
                                "gemma2_9b", "gemma2_27b", 
                                "mixtral_8x7b", "mixtral_8x22b",
                                #"sqlcoder:7b", "sqlcoder:15b", "sqlcoder:70b"
                               ],
                    "openai_chat" : ["gpt-3.5-turbo", "gpt-4o", "gpt-4o-mini"]
                   }

for q in df_question.index:
    question_id = df_question.loc[q, "id"]
    for iteration in [1, 2, 3]:
        for llm_type, models in avalaible_models.items():
            for model in models:
                model.replace(":", "_")
                index_name = f"Q_{str(question_id).zfill(2)}__{llm_type}__{model}__RAG_benchmark__iteration_{iteration}"
                if index_name not in df.index:
                    for col in df_question.columns:
                        value = df_question.loc[q, col]
                        if col == "sql":
                            col = "answer_sql_expected"
                        elif col == "answer":
                            col = "answer_summary_expected"
                        df.loc[index_name, col] = value
                        df.loc[index_name, "chatbgc_version"] = chatbgc_version
                        df.loc[index_name, "benchmark_version"] = benchmark_version
                        df.loc[index_name, "model"] = model
                        df.loc[index_name, "llm_type"] = llm_type
                        df.loc[index_name, "sql_query_success"] = False

In [ ]:
df.to_csv("1.Refined_benchmark_response.tsv", sep="\t")

In [ ]:
df_evaluation

In [ ]:
with open(f"../result/gpt-4o/chatbgc_{chatbgc_version}/benchmark_{benchmark_version}/iteration_2/benchmark.log", "r") as f:
    benchmark_log = f.readlines()

In [ ]:
log_data = {}
ctr = 0
for num, line in enumerate(benchmark_log):
    if 'INFO - Processing question' in line:
        print("\n-----------")
        print(num, line)
        start = num
        ctr += 1
    if 'processed successfully' in line:
        print(num, line)
        stop = num
        log_data[ctr] = [start, stop]
        

In [ ]:
log_data

In [ ]:
q = 1
benchmark_log[log_data[q][0]:log_data[q][1]+1]

In [ ]:
q = 3
benchmark_log[log_data[q][0]:log_data[q][1]+1]